In [1]:
import sys
sys.path.append("/projectnb/cs640/students/endrit01/mt")   # modify this path

from AI_thrember.model import read_vectorized_features
import lightgbm as lgb

data_dir = "/projectnb/cs640/students/endrit01/mt/data"

/projectnb/cs640/students/endrit01/.conda/envs/aimal-310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from AI_thrember.model import read_vectorized_features
import inspect
print(inspect.getsource(read_vectorized_features))

def read_vectorized_features(data_dir: Path | str, subset: str = "train"):
    """
    Read vectorized features using numpy.memmap WITHOUT loading into RAM.
    """
    data_path = Path(data_dir)
    X_path = data_path / f"X_{subset}.dat"
    y_path = data_path / f"y_{subset}.dat"

    if not X_path.is_file():
        raise ValueError(f"Invalid subset file: {X_path}")
    if not y_path.is_file():
        raise ValueError(f"Invalid subset file: {y_path}")

    extractor = PEFeatureExtractor()
    ndim = extractor.dim

    # Determine number of rows in X via file size
    bytes_per_row = ndim * 4   # float32 = 4 bytes
    total_bytes = os.path.getsize(X_path)
    nrows = total_bytes // bytes_per_row

    # Memory-map WITHOUT loading into RAM
    X = np.memmap(X_path, dtype=np.float32, mode="r", shape=(nrows, ndim))
    
    # y is int32 so 4 bytes each
    y_nbytes = os.path.getsize(y_path)
    y_rows = y_nbytes // 4

    # Multilabel case: shape is (nrows, n_labels)
    if y_rows == nro

In [3]:
import os

# Show hostname of the machine running this notebook
!hostname

# Show the username and current node
print("User:", os.getlogin())
print("Node:", os.uname().nodename)


scc-fi3


OSError: [Errno 6] No such device or address

In [3]:
import os
import multiprocessing
import time
from AI_thrember.model import create_vectorized_features

# ================================================
# 1. Detect and use ALL CPU cores allocated by SCC
# ================================================
num_cores = multiprocessing.cpu_count()
print(f"Detected CPU cores: {num_cores}")

# Force Python + LightGBM + BLAS to use all cores
os.environ["OMP_NUM_THREADS"] = str(num_cores)
os.environ["OPENBLAS_NUM_THREADS"] = str(num_cores)
os.environ["MKL_NUM_THREADS"] = str(num_cores)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(num_cores)
os.environ["NUMEXPR_NUM_THREADS"] = str(num_cores)

print("Thread environment configured.")

# ================================================
# 2. Set your dataset directory on SCC
# ================================================
data_dir = "/projectnb/cs640/students/endrit01/mt/data"
print("Dataset path:", data_dir)

# ================================================
# 3. Run FAMILY vectorization
# ================================================
print("\n===== STARTING FAMILY VECTORIZATION =====")
start = time.time()

create_vectorized_features(
    data_dir=data_dir,
    label_type="family",   # MULTICLASS
    class_min=1200           # keep only families with >=40 samples
)

end = time.time()
print("\n===== DONE =====")
print(f"Total time: {(end - start)/60:.2f} minutes")


Detected CPU cores: 32
Thread environment configured.
Dataset path: /projectnb/cs640/students/endrit01/mt/data

===== STARTING FAMILY VECTORIZATION =====
Preparing to vectorize raw features


100%|██████████| 5252000/5252000 [09:09<00:00, 9555.82it/s] 


Vectorizing training set


100%|██████████| 1212000/1212000 [02:10<00:00, 9319.75it/s] 


Vectorizing test set


100%|██████████| 1212000/1212000 [06:27<00:00, 3128.53it/s]


Vectorizing challenge set


100%|██████████| 6315/6315 [00:02<00:00, 2384.70it/s]


===== DONE =====
Total time: 46.50 minutes


In [6]:
from AI_thrember.model import read_vectorized_features

X, y = read_vectorized_features("/projectnb/cs640/students/endrit01/mt/data", "train")
print(X.shape)
print(y.shape)

(5252000, 2568)
(5252000,)


In [4]:
X, y = read_vectorized_features("/projectnb/cs640/students/endrit01/mt/data", "train")

print("X:", X.shape)
print("y:", y.shape)
print("Unique classes:", len({label for labels in y for label in (labels if isinstance(labels, list) else [labels])}))


X: (5252000, 2568)
y: (5252000,)
Unique classes: 223


In [8]:
import json

path = "/projectnb/cs640/students/endrit01/mt/data/2024-11-24_2024-11-30_Win32_test.jsonl"

with open(path) as f:
    first = json.loads(next(f))

print(first["family"])
print(type(first["family"]))

rungbu
<class 'str'>


In [12]:
import numpy as np
y = np.memmap("/projectnb/cs640/students/endrit01/mt/data/ttest/y_train.dat", dtype=np.float32, mode="r")
print(np.unique(y)[:50])

[1.4e-45 2.8e-45 4.2e-45 5.6e-45 7.0e-45 8.4e-45 9.8e-45 1.1e-44 1.3e-44
 1.4e-44 1.5e-44 1.7e-44 1.8e-44 2.0e-44 2.1e-44 2.2e-44 2.4e-44 2.5e-44
 2.7e-44 2.8e-44 2.9e-44 3.1e-44 3.2e-44 3.4e-44 3.5e-44 3.6e-44 3.8e-44
 3.9e-44 4.1e-44 4.2e-44 4.3e-44 4.5e-44 4.6e-44 4.8e-44 4.9e-44 5.0e-44
 5.2e-44 5.3e-44 5.5e-44 5.6e-44 5.7e-44 5.9e-44 6.0e-44 6.2e-44 6.3e-44
 6.4e-44 6.6e-44 6.7e-44 6.9e-44 7.0e-44]


In [7]:
import json

path = "/projectnb/cs640/students/endrit01/mt/data/2024-11-24_2024-11-30_Win32_test.jsonl"

families = []
count = 0

with open(path) as f:
    for line in f:
        obj = json.loads(line)
        fam = obj.get("family", None)
        families.append(fam)
        count += 1
        if count >= 200:   # inspect first 200 samples
            break

print("Unique families in first 200:", set(families))
print("Counts:", {k: families.count(k) for k in set(families)})


Unique families in first 200: {'upatre', None, 'injurer', 'rugmi', 'innobundle', 'sality', 'berbew', 'titirez', 'diztakun', 'shodi', 'cayunamer', 'njrat', 'dumpex', 'cosmu', 'rungbu', 'scarletflash', 'chewbacca', 'cryptowall', 'allaple', 'lolbot', 'chinky', 'avrora', 'tinyloader', 'tepfer', 'neshta', 'wacatac', 'offloader', 'snackarcin', 'swisyn', 'ming', 'amadey', 'yomal', 'ipamor', 'shandagames', 'mofksys', 'puce', 'mydoom', 'convagent', 'vidar', 'aenjaris', 'softcnapp', 'barys', 'shutdown', 'axespec', 'lummastealer', 'vbclone', 'blihan', 'triusor', 'stealc', 'zeus', 'loudoffer', 'anyplacecontrol', 'bancteian', 'remcos'}
Counts: {'upatre': 1, None: 122, 'injurer': 1, 'rugmi': 1, 'innobundle': 1, 'sality': 1, 'berbew': 20, 'titirez': 1, 'diztakun': 1, 'shodi': 1, 'cayunamer': 1, 'njrat': 1, 'dumpex': 1, 'cosmu': 3, 'rungbu': 1, 'scarletflash': 1, 'chewbacca': 1, 'cryptowall': 1, 'allaple': 1, 'lolbot': 1, 'chinky': 2, 'avrora': 1, 'tinyloader': 1, 'tepfer': 1, 'neshta': 2, 'wacatac': 